# Monitorear modelo desplegado

In [ ]:
! pip show azure-ai-ml

## Conectar a workspace

In [ ]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

## Obtener datos de test

In [ ]:
import pandas as pd

data_asset = ml_client.data.get("<diabetes-data>", version="1")
df = pd.read_csv(data_asset.path)

x = df.drop(columns=["Diabetic"])
y = df["Diabetic"]

# convertir a JSON
x_real = x[:6]
y_real = y[:6]


## Obtener predicción del deploymnent

In [ ]:
# Guardar como json
import json

data = {
  "input_data": {
    "columns": x_real.columns.tolist(),
    "index": [x_real.index.tolist()],
    "data": x_real.values.tolist()
  }
}


print(data)


with open('sample-data.json', 'w') as fp:
    json.dump(data, fp)

In [ ]:

# Invocar endpoint
y_pred_str = ml_client.online_endpoints.invoke(
    endpoint_name='<endpoint-name>',
    deployment_name="blue",
    request_file="sample-data.json",
    
)

# converir a lista
import ast
y_pred = ast.literal_eval(y_pred_str)

print(y_pred)

## Monitoreo del Rendimiento del Modelo

In [ ]:
from sklearn.metrics import accuracy_score, precision_score
import mlflow

# Simular datos reales
y_true = y_real.values.tolist()

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)

# Registrar métricas en Azure ML
with mlflow.start_run():
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)

## Detección de Deriva de Datos

In [ ]:
from azure.ai.ml.entities import MonitorDefinition, MonitoringTarget, AlertNotification
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Configurar cliente de Azure ML
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# Definir el objetivo de monitoreo
monitoring_target = MonitoringTarget(
    ml_task="classification",
    endpoint_deployment_id="azureml:credit-default:main"
)

# Crear alertas
alert_notification = AlertNotification(emails=["admin@example.com"])

# Definir el monitor
monitor_definition = MonitorDefinition(
    compute="cpu-cluster",
    monitoring_target=monitoring_target,
    alert_notification=alert_notification
)

# Crear el monitor
model_monitor = ml_client.model_monitors.create_or_update(monitor_definition)

## Configuración de alertas

In [ ]:
from azure.ai.ml.entities import AlertNotification

# Configurar alertas por correo electrónico
alert_notification = AlertNotification(emails=["admin@example.com"])
monitor_definition.alert_notification = alert_notification

## Pipeline de retraining

In [ ]:

from azure.ai.ml import command, Input
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Configurar cliente de Azure ML
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

# Definir el job de reentrenamiento
retrain_job = command(
    code="./src",
    command="python train.py --training_data ${{inputs.training_data}}",
    inputs={
        "training_data": Input(type="uri_folder", path="azureml://datastores/my_datastore/paths/new_training_data.csv"),
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="my-cluster",
)

# Enviar el job de reentrenamiento
returned_job = ml_client.jobs.create_or_update(retrain_job)
print(f"Job de reentrenamiento enviado: {returned_job.name}")